# Styling Options

:::{note}
These are the styling options shared by all the supported plotting backends. Backend-specific options can be found on the reference page of each plotting method (e.g. {meth}`hvplot.hvPlot.scatter`) or by executing `hvplot.help('scatter', docstring=False, generic=False, style=True)`.
:::

Visual styling options to adjust colors, fonts, and other aesthetic elements of the plot:

```{eval-rst}
.. plotting-options-table:: Styling Options
```

(option-backend_opts)=
## `backend_opts`

hvPlot offers many options to customize a plot. Yet, sometimes this is not enough, in which case `backend_opts` offers a powerful mechanism to customize many properties of the plot objects created internally by HoloViews (Bokeh models, Matplotlib figure and axis, etc.). This option accepts a dictionary whose keys are string accessors, i.e. they mirror attribute access on these plot objects. The values of this dictionary are the values to set.

The objects that can be targeted via this accessor pattern are those referenced in the HoloViews plot `handles` dictionary. Usually that would be the `plot` key for Bokeh giving access to the [`figure`](https://docs.bokeh.org/en/latest/docs/reference/plotting/figure.html) object, and the `fig` key for Matplotlib giving access to the [`Figure`](https://matplotlib.org/stable/api/_as_gen/matplotlib.figure.Figure) object. Note that accessors starting with `plot` for Bokeh and `fig` for Matplotlib can omit these terms (e.g. `xgrid.grid_line_color` instead of `plot.xgrid.grid_line_colore` for Bokeh).

Let's start with a Bokeh example where we set properties of the grid, the x- and y-axis, and hover. These options cannot be directly customized with the current hvPlot's API.

In [ ]:
import hvplot.pandas  # noqa

df = hvplot.sampledata.penguins("pandas")

plot = df.hvplot.scatter(
    x="bill_length_mm", y="bill_depth_mm", color="black",
    backend_opts={
        "plot.xgrid.grid_line_color": "grey",
        "plot.xgrid.grid_line_dash": "dotted",
        "plot.xaxis.axis_label_text_font_style": "bold",
        "plot.yaxis.axis_label_text_font_style": "bold",
        "hover.attachment": "vertical",
    }
)
plot

With the code below, we can see all the handles we could have referenced for *this plot* (other plots may have different handles).

In [ ]:
import holoviews as hv

hv.renderer('bokeh').get_plot(plot).handles

We can see the Bokeh property `grid_line_color` exists and has been correctly set.

In [ ]:
hv.renderer('bokeh').get_plot(plot).handles['plot'].xgrid.grid_line_color

In this Matplotlib example we disable the plot frame. Note that we could also have written the accessor as `'axes.set_frame_on'` (which is the method called internally by HoloViews).

In [ ]:
import hvplot.pandas  # noqa
hv.extension("matplotlib")

df = hvplot.sampledata.penguins("pandas")

plot = df.hvplot.scatter(
    x="bill_length_mm", y="bill_depth_mm",
    backend_opts={"axes.frame_on": False}
)
plot

Here's the list of handles for *this plot*.

In [ ]:
hv.renderer('matplotlib').get_plot(plot).handles

We can see the Matplotlib method `set_frame_on` exists and has been correctly executed.

In [ ]:
hv.renderer('matplotlib').get_plot(plot).handles['axis'].set_frame_on

In [ ]:
hv.renderer('matplotlib').get_plot(plot).handles['axis'].get_frame_on()

:::{seealso}
The [`legend_opts`](option-legend_opts) option to customize specifically the styling of the legend.
:::

(option-fontscale)=
## `fontscale`

The `fontscale` option scales all the fonts in the plot by the provided numeric factor. For example, setting `fontscale=1.5` enlarges the title, tick labels, and axis labels by 50%. This is useful when you want to emphasize text for presentations or detailed viewing.

In [ ]:
import hvplot.pandas  # noqa

df = hvplot.sampledata.penguins("pandas")

df.hvplot.scatter(
    x='bill_length_mm', y='bill_depth_mm', by='species',
    fontscale=1.5, title="Penguins Species",
)

(option-fontsize)=
## `fontsize`

The `fontsize` option sets the font size for different text elements in the plot. It can be:

- A single value (e.g. `12`) to apply to all text elements.
- A dictionary to control specific elements like the title, axis labels, and ticks.  
  Example: `{'title': '15pt', 'xlabel': '12pt', 'ylabel': '12pt', 'ticks': 10}`

This option is useful for precise text control in reports, presentations, and dashboards.

::: {note}
**Backend-specific behavior**

- The **Bokeh** backend supports both numeric values and strings with units like `'12pt'` or `'10px'`.
- The **Matplotlib** backend only accepts **numeric values**. Using strings like `'14pt'` will not work.

To ensure compatibility across backends, prefer using numbers unless you're specifically targeting Bokeh.
:::

**Bokeh backend**

In [ ]:
import hvplot.pandas  # noqa

df = hvplot.sampledata.penguins("pandas")

df.hvplot.scatter(
    x='bill_length_mm', y='bill_depth_mm', by='species',
    fontsize={'title': '16pt', 'xlabel': '12pt', 'ylabel': '14pt', 'ticks': 10},
    title="Penguins Measurements by Species"
)

**Matplotlib backend**

In [ ]:
import hvplot.pandas  # noqa
hvplot.extension('matplotlib')

df = hvplot.sampledata.penguins("pandas")

df.hvplot.scatter(
    x='bill_length_mm', y='bill_depth_mm', by='species',
    fontsize={'title': 22, 'xlabel': 16, 'ylabel': 18, 'ticks': 12},
    title="Penguins Measurements by Species"
)

(option-grid)=
## `grid`

Turns grid lines on or off behind your data. By default grids are disabled; use `grid=True` to draw horizontal and vertical lines at each major tick mark.

In [ ]:
import hvplot.pandas  # noqa

df = hvplot.sampledata.stocks("pandas", engine_kwargs={"index_col" : "date"})

plot1 = df.hvplot(group_label="Company", width=400, title="Default: grid=False")
plot2 = df.hvplot(group_label="Company", grid=True, width=400, title="grid=True")
(plot1 + plot2).cols(1)